In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas_ta as ta
import yfinance as yf 
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

In [2]:
df = pd.read_csv('2022.2.9BTCUSDT-H1-NoSession.csv')
df.Date = df.Date.astype(str)+' '+ df.Time
df.Date = pd.to_datetime(df.Date)
df.drop(columns= ['Time'], inplace = True)
df = df.set_index('Date')

In [3]:
#initial
def set_up_df(df:pd.DataFrame , index):  
    df1    =   pd.DataFrame(columns=index)
    df1['date']  =    df.index 
    df1.date = pd.to_datetime(df1.date,format='%Y-%m-%d')
    df1 = df1.set_index('date')
    return df1

#for calculate duration
def convert_to_hours(delta):
    total_seconds = delta.total_seconds()
    hours = str(int(total_seconds // 3600))
    return hours

#Performance Matric
def cal_consecutive(df:pd.DataFrame):
    df['win']= np.sign(df) # assign sign on log
    blocks = df['win'].ne(df['win'].shift()).cumsum() 
    df['consecutive_WL'] = df['win'].groupby(blocks).cumsum()
    return df['consecutive_WL'] 

def Cal_portstat(port):
    port_return = port.total_return().to_frame()
    port_return['MaxDD'] = port.max_drawdown()
    port_return['NumTrade'] = port.trades.count()
    port_return['WinRate'] = port.trades.win_rate()
    port_return['AvgWinningTrade'] = port.trades.winning.returns.mean()
    port_return['AvgLosingTrade'] = port.trades.losing.returns.mean()
    port_return['SharpRatio'] = port.returns_acc.sharpe_ratio()
    return port_return

def Cal_pip(df):
    pip = []
    pipp = df['Avg Exit Price'] - df['Avg Entry Price']
    for i in portfinal['Direction']:
        if i == 'Long':
            pip.append(1)
        else :
            pip.append(-1)    
    return (pip * pipp) 

def Cal_mae_mfe(data,port):
    MAE = []
    MFE = []
    for i,j,y,k in zip(port['Entry Timestamp'], port['Exit Timestamp'], port['Avg Entry Price'], port['Direction']):
        if k == 'Long':
            MFE.append((data.loc[i:j]['High'].max() - y)/y)
            MAE.append((y - df.loc[i:j]['Low'].min())/y)
        else :
            MAE.append((data.loc[i:j]['High'].max() - y)/y)
            MFE.append((y - df.loc[i:j]['Low'].min())/y)
    return MAE, MFE  

def Calkelly(port):
    init_cash = 10000
    W = port.stats()[15]/100
    A = abs(port.stats()[19])
    B = port.stats()[18]
    Percent_kelly = (W/A) - ((1-W)/B)
    riskpertrade = Percent_kelly * init_cash
    print(f'Percent_kelly = {Percent_kelly}')

#### Default Supertrend

In [4]:
supert = df.ta.supertrend(7)['SUPERTd_7_3.0']

long_signal = (supert == 1) # & (adx<30)
short_signal = (supert == -1) #& (adx<30)
  
long_exit = short_signal
short_exit = long_signal

#add to Empty Dataframe
LongEntries = long_signal
LongExit= long_exit    
ShortEntries= short_signal
ShortExit= short_exit

In [5]:
port = vbt.Portfolio.from_signals(df.Close,
                                entries = LongEntries,
                                exits = LongExit,
                                short_entries = ShortEntries,
                                short_exits = ShortExit,
                                freq = 'H',
                                init_cash = 10000,
                                fees = 0.00,
                                slippage = 0.00)

In [6]:
Calkelly(port)

Percent_kelly = 0.03637754486842301


In [7]:
port.stats()

Start                               2017-08-17 04:00:00
End                                 2022-02-03 00:00:00
Period                               1625 days 15:00:00
Start Value                                     10000.0
End Value                                 223127.228917
Total Return [%]                            2131.272289
Benchmark Return [%]                         755.879438
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              54.732545
Max Drawdown Duration                 211 days 08:00:00
Total Trades                                        704
Total Closed Trades                                 703
Total Open Trades                                     1
Open Trade PnL                              4779.603853
Win Rate [%]                                   39.97155
Best Trade [%]                                59.564738
Worst Trade [%]                              -18

In [8]:
port.value().vbt.plot() #equity curve

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': 'cc697…

#### Optimise SL

In [9]:
df['atr'] = df.ta.atr()
portstats = []
start, end, step = (1, 4, 0.1)
array = np.arange(start,end,step)
for i in array:
    multiply = i
    df['sl_stop'] = (df['atr'] * multiply) / df.Close
    port = vbt.Portfolio.from_signals(df.Close,
                                    entries = LongEntries,
                                    exits = LongExit,
                                    short_entries = ShortEntries,
                                    short_exits = ShortExit,
                                    freq = 'H',
                                    init_cash = 10000,
                                    fees = 0.00,
                                    slippage = 0.00,
                                    sl_stop = df['sl_stop']  
                                     )
    portstats.append(port.stats())
pd.DataFrame(portstats).sort_values('Sharpe Ratio',ascending=False).head(5)    

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
3,2017-08-17 04:00:00,2022-02-03,1625 days 15:00:00,10000.0,331239.807710,3212.398077,755.879438,100.0,0.0,51.794364,...,5.719103,-1.870047,3 days 09:35:12.302839116,0 days 17:38:52.678821879,1.109118,307.214610,1.389927,2.306046,1.050568,2.036698
2,2017-08-17 04:00:00,2022-02-03,1625 days 15:00:00,10000.0,290653.438577,2806.534386,755.879438,100.0,0.0,52.348659,...,5.688523,-1.821352,3 days 09:12:22.500000,0 days 16:42:33.432032301,1.109208,260.050696,1.353106,2.160390,1.049235,1.981322
7,2017-08-17 04:00:00,2022-02-03,1625 days 15:00:00,10000.0,247389.310527,2373.893105,755.879438,100.0,0.0,56.714750,...,5.825093,-2.125084,3 days 11:52:53.684210526,0 days 21:02:18.906752411,1.089220,252.481476,1.304581,1.860542,1.047340,1.908592
1,2017-08-17 04:00:00,2022-02-03,1625 days 15:00:00,10000.0,242713.841846,2327.138418,755.879438,100.0,0.0,49.594705,...,5.611094,-1.747879,3 days 08:23:37.846153846,0 days 15:31:33.773824650,1.109118,206.106344,1.301894,2.109935,1.047406,1.901948
4,2017-08-17 04:00:00,2022-02-03,1625 days 15:00:00,10000.0,242796.684640,2327.966846,755.879438,100.0,0.0,53.833166,...,5.751353,-1.966473,3 days 09:56:55.974440894,0 days 18:40:52.401746724,1.096720,229.271894,1.300595,1.944104,1.047246,1.903075


#### implove with  SL

In [10]:
df['atr'] = df.ta.atr()
df['sl_stop'] = (df['atr'] * 1.3) / df.Close
port = vbt.Portfolio.from_signals(df.Close,
                                entries = LongEntries,
                                exits = LongExit,
                                short_entries = ShortEntries,
                                short_exits = ShortExit,
                                freq = 'H',
                                init_cash = 10000,
                                fees = 0.00,
                                slippage = 0.00, sl_stop = df['sl_stop'])

In [11]:
Calkelly(port)

Percent_kelly = 0.043538433185641434


In [12]:
port.stats()

Start                               2017-08-17 04:00:00
End                                 2022-02-03 00:00:00
Period                               1625 days 15:00:00
Start Value                                     10000.0
End Value                                  331239.80771
Total Return [%]                            3212.398077
Benchmark Return [%]                         755.879438
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              51.794364
Max Drawdown Duration                 208 days 00:00:00
Total Trades                                       1031
Total Closed Trades                                1030
Total Open Trades                                     1
Open Trade PnL                              4808.759645
Win Rate [%]                                  30.776699
Best Trade [%]                                62.389889
Worst Trade [%]                               -9

In [13]:
port.value().vbt.plot() #equity curve

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': '58400…

### improve with MA

In [14]:
supert = df.ta.supertrend(7)['SUPERTd_7_3.0']
ema1 = df.ta.ema(50)
ema2 = df.ta.ema(200)

long_signal = (supert == 1)  & (ema1>ema2)
short_signal = (supert == -1) & (ema1<ema2)
  
long_exit = (supert == -1)
short_exit = (supert == 1)

#add to Empty Dataframe
LongEntries = long_signal
LongExit= long_exit    
ShortEntries= short_signal
ShortExit= short_exit

In [15]:
port = vbt.Portfolio.from_signals(df.Close,
                                entries = LongEntries,
                                exits = LongExit,
                                short_entries = ShortEntries,
                                short_exits = ShortExit,
                                freq = 'H',
                                init_cash = 10000,
                                fees = 0.00,
                                slippage = 0.00)

In [16]:
Calkelly(port)

Percent_kelly = 0.0435048966005937


In [17]:
port.stats()

Start                               2017-08-17 04:00:00
End                                 2022-02-03 00:00:00
Period                               1625 days 15:00:00
Start Value                                     10000.0
End Value                                 122221.830656
Total Return [%]                            1122.218307
Benchmark Return [%]                         755.879438
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              43.857316
Max Drawdown Duration                 395 days 16:00:00
Total Trades                                        453
Total Closed Trades                                 453
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  38.410596
Best Trade [%]                                55.417701
Worst Trade [%]                              -13

In [18]:
port.value().vbt.plot() #equity curve

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': 'e1feb…

#### Improve with MA + SL

In [19]:
supert = df.ta.supertrend(7)['SUPERTd_7_3.0']
ema1 = df.ta.ema(50)
ema2 = df.ta.ema(200)

long_signal = (supert == 1)  & (ema1>ema2)
short_signal = (supert == -1) & (ema1<ema2)
  
long_exit = (supert == -1)
short_exit = (supert == 1)

#add to Empty Dataframe
LongEntries = long_signal
LongExit= long_exit    
ShortEntries= short_signal
ShortExit= short_exit

df['atr'] = df.ta.atr()
df['sl_stop'] = (df['atr'] * 1.3) / df.Close
port = vbt.Portfolio.from_signals(df.Close,
                                entries = LongEntries,
                                exits = LongExit,
                                short_entries = ShortEntries,
                                short_exits = ShortExit,
                                freq = 'H',
                                init_cash = 10000,
                                fees = 0.00,
                                slippage = 0.00, sl_stop = df['sl_stop'])

In [20]:
Calkelly(port)

Percent_kelly = 0.04376677444422804


In [21]:
port.stats()

Start                               2017-08-17 04:00:00
End                                 2022-02-03 00:00:00
Period                               1625 days 15:00:00
Start Value                                     10000.0
End Value                                 104735.954669
Total Return [%]                             947.359547
Benchmark Return [%]                         755.879438
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              42.983094
Max Drawdown Duration                 390 days 18:00:00
Total Trades                                        657
Total Closed Trades                                 657
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  29.984779
Best Trade [%]                                55.417701
Worst Trade [%]                               -9

In [22]:
port.value().vbt.plot() #equity curve

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': '3cff8…

#### Improve with EMA, ADX, SL

In [23]:
supert = df.ta.supertrend(7)['SUPERTd_7_3.0']
adx = df.ta.adx()['ADX_14']
ema1 = df.ta.ema(50)
ema2 = df.ta.ema(200)
long_signal = (supert == 1)  & (adx>20)& (ema1>ema2)
short_signal = (supert == -1) & (adx>20) & (ema1<ema2)
  
long_exit = (supert == -1)
short_exit = (supert == 1)

#add to Empty Dataframe
LongEntries = long_signal
LongExit= long_exit    
ShortEntries= short_signal
ShortExit= short_exit

df['atr'] = df.ta.atr()
df['sl_stop'] = (df['atr'] * 1.3) / df.Close
port = vbt.Portfolio.from_signals(df.Close,
                                entries = LongEntries,
                                exits = LongExit,
                                short_entries = ShortEntries,
                                short_exits = ShortExit,
                                freq = 'H',
                                init_cash = 10000,
                                fees = 0.00,
                                slippage = 0.00, sl_stop = df['sl_stop'])
Calkelly(port)

Percent_kelly = 0.045941130574704184


In [24]:
port.stats()

Start                               2017-08-17 04:00:00
End                                 2022-02-03 00:00:00
Period                               1625 days 15:00:00
Start Value                                     10000.0
End Value                                  97870.434936
Total Return [%]                             878.704349
Benchmark Return [%]                         755.879438
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              40.832657
Max Drawdown Duration                 390 days 18:00:00
Total Trades                                        594
Total Closed Trades                                 594
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                   30.13468
Best Trade [%]                                55.417701
Worst Trade [%]                               -9

In [25]:
port.value().vbt.plot() #equity curve

FigureWidget({
    'data': [{'showlegend': False,
              'type': 'scatter',
              'uid': 'bf7b5…